# SQL 的五十道練習

> 垂直與水平合併查詢結果 

[數據交點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

In [1]:
import sqlite3
import unittest
import numpy as np
import pandas as pd
conn = sqlite3.connect('../databases/nba.db')
conn.execute("""ATTACH '../databases/covid19.db' AS covid19""")
conn.execute("""ATTACH '../databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH '../databases/imdb.db' AS imdb""")

## 結合 `twElection2020` 資料庫中的三個資料表：`legislative_at_large`、`legislative_regional` 與 `presidential` 查詢親民黨、中國國民黨與民主進步黨在總統副總統、不分區立委與區域立委的得票率，將得票率以 `ROUND()` 函數取至小數點後第四位，並且將查詢結果依衍生計算欄位 `election` 遞增排序。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(9, 3) 的查詢結果。

```
   party election  votes_percentage
0  中國國民黨    不分區立委            0.3336
1  民主進步黨    不分區立委            0.3398
2    親民黨    不分區立委            0.0366
3  中國國民黨     區域立委            0.4071
4  民主進步黨     區域立委            0.4511
5    親民黨     區域立委            0.0043
6  中國國民黨    總統副總統            0.3861
7  民主進步黨    總統副總統            0.5713
8    親民黨    總統副總統            0.0426
```

In [2]:
summarize_party_votes_percentages =\
"""
-- SQL 查詢語法起點
SELECT CASE WHEN number = 1 THEN '親民黨'
            WHEN number = 2 THEN '中國國民黨'
            ELSE '民主進步黨'
       END AS party,
       '總統副總統' AS election,
       ROUND(CAST(SUM(votes) AS REAL) / (SELECT SUM(votes) FROM presidential), 4) AS votes_percentage
  FROM presidential
 GROUP BY number
 UNION
SELECT party,
       '不分區立委' AS election,
       ROUND(CAST(SUM(votes) AS REAL) / (SELECT SUM(votes) FROM legislative_at_large), 4) AS votes_percentage
  FROM legislative_at_large
 WHERE party IN ('親民黨', '中國國民黨', '民主進步黨')
 GROUP BY party
 UNION
SELECT party,
       '區域立委' AS election,
       ROUND(CAST(SUM(votes) AS REAL) / (SELECT SUM(votes) FROM legislative_regional), 4) AS votes_percentage
  FROM legislative_regional
 WHERE party IN ('親民黨', '中國國民黨', '民主進步黨')
 GROUP BY party
 ORDER BY election;
-- SQL 查詢語法終點
"""

## 結合 `nba` 資料庫中的三個資料表：`career_summaries`、`players` 與 `teams` 查詢洛杉磯湖人隊球員的生涯場均得分 `ppg`，將查詢結果依 `ppg` 遞減排序。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(16, 4) 的查詢結果。

```
              fullName   firstName       lastName   ppg
0   Los Angeles Lakers      LeBron          James  27.0
1   Los Angeles Lakers     Anthony          Davis  23.9
2   Los Angeles Lakers        Kyle          Kuzma  15.4
3   Los Angeles Lakers        Marc          Gasol  14.2
4   Los Angeles Lakers      Dennis       Schroder  14.1
5   Los Angeles Lakers    Montrezl        Harrell  12.9
6   Los Angeles Lakers      Wesley       Matthews  12.8
7   Los Angeles Lakers  Kentavious  Caldwell-Pope  11.4
8   Los Angeles Lakers    Markieff         Morris  11.1
9   Los Angeles Lakers       Talen  Horton-Tucker   7.4
10  Los Angeles Lakers       Jared         Dudley   7.3
11  Los Angeles Lakers        Alex         Caruso   5.7
12  Los Angeles Lakers      Damian          Jones   4.3
13  Los Angeles Lakers     Alfonzo       McKinnie   4.0
14  Los Angeles Lakers    Devontae          Cacok   2.6
15  Los Angeles Lakers      Kostas  Antetokounmpo   1.2
```

In [3]:
list_lakers_players_ppg =\
"""
-- SQL 查詢語法起點
SELECT teams.fullName,
       players.firstName,
       players.lastName,
       career_summaries.ppg
  FROM players
  JOIN teams
    ON players.teamId = teams.teamId
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 WHERE teams.nickname = 'Lakers'
 ORDER BY ppg DESC;
-- SQL 查詢語法終點
"""

## 結合 `nba` 資料庫中的三個資料表：`career_summaries`、`players` 與 `teams` 查詢 30 支球隊的得分王（生涯場均得分 `ppg` 全隊最高），將查詢結果依隊伍名 `fullName` 排序。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(30, 3) 的查詢結果。

```
                      team                   player   ppg
0            Atlanta Hawks               Trae Young  24.1
1           Boston Celtics             Kemba Walker  19.8
2            Brooklyn Nets             Kevin Durant  27.1
3        Charlotte Hornets              LaMelo Ball  15.7
4            Chicago Bulls              Zach LaVine  18.7
5      Cleveland Cavaliers            Collin Sexton  19.5
6         Dallas Mavericks              Luka Doncic  25.5
7           Denver Nuggets             Nikola Jokic  17.9
8          Detroit Pistons             Josh Jackson  12.2
9    Golden State Warriors            Stephen Curry  23.8
10         Houston Rockets                John Wall  19.1
11          Indiana Pacers              T.J. Warren  15.5
12             LA Clippers              Paul George  20.1
13      Los Angeles Lakers             LeBron James  27.0
14       Memphis Grizzlies                Ja Morant  18.3
15              Miami Heat             Jimmy Butler  17.2
16         Milwaukee Bucks    Giannis Antetokounmpo  20.7
17  Minnesota Timberwolves       Karl-Anthony Towns  22.6
18    New Orleans Pelicans          Zion Williamson  24.4
19         New York Knicks             Derrick Rose  18.6
20   Oklahoma City Thunder  Shai Gilgeous-Alexander  16.2
21           Orlando Magic           Nikola Vucevic  16.6
22      Philadelphia 76ers              Joel Embiid  24.7
23            Phoenix Suns             Devin Booker  22.7
24  Portland Trail Blazers           Damian Lillard  24.6
25        Sacramento Kings             De'Aaron Fox  17.4
26       San Antonio Spurs            DeMar DeRozan  20.1
27         Toronto Raptors               Kyle Lowry  14.9
28               Utah Jazz         Donovan Mitchell  22.9
29      Washington Wizards        Russell Westbrook  23.2
```

In [4]:
list_ppg_leader_by_teams =\
"""
-- SQL 查詢語法起點
SELECT teams.fullName AS team,
       players.firstName || ' ' || players.lastName AS player,
       MAX(career_summaries.ppg) AS ppg
  FROM players
  JOIN teams
    ON players.teamId = teams.teamId
  JOIN career_summaries
    ON players.personId = career_summaries.personId
 GROUP BY teams.fullName
 ORDER BY teams.fullName;
-- SQL 查詢語法終點
"""

## 結合 `imdb` 資料庫中的三個資料表：`actors`、`casting` 與 `top_rated_movies` 查詢 Tom Hanks 在 IMDb 最高評價的 250 部電影中有在哪幾部演出，依據 `casting` 資料表中的 `ord` 註記是否為第一主角（`ord` 若為 1 表示為第一主角），將查詢結果依 `release_year` 遞增排序。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(6, 5) 的查詢結果。

```
   release_year                title  rating       name  is_lead_actor
0          1994         Forrest Gump     8.8  Tom Hanks              1
1          1995            Toy Story     8.3  Tom Hanks              1
2          1998  Saving Private Ryan     8.6  Tom Hanks              1
3          1999       The Green Mile     8.6  Tom Hanks              1
4          2002  Catch Me If You Can     8.1  Tom Hanks              0
5          2010          Toy Story 3     8.2  Tom Hanks              1
```

In [5]:
list_movies_in_which_tom_hanks_appeared =\
"""
-- SQL 查詢語法起點
SELECT movies.release_year,
       movies.title,
       movies.rating,
       actors.name,
       CASE WHEN casting.ord = 1 THEN 1
            ELSE 0
       END as is_lead_actor
  FROM actors
  JOIN casting
    ON actors.id = casting.actor_id
  JOIN top_rated_movies AS movies
    ON casting.movie_id = movies.id
 WHERE actors.name = 'Tom Hanks'
 ORDER BY movies.release_year;
-- SQL 查詢語法終點
"""

## 結合 `imdb` 資料庫中的三個資料表：`actors`、`casting` 與 `top_rated_movies` 查詢在 IMDb 最高評價的 250 部電影中有哪些作品是有 Leonardo DiCaprio 演出或由 Christopher Nolan 擔任導演，將查詢結果依 `release_year` 與 `title` 遞增排序。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(12, 3) 的查詢結果。

```
    release_year                    title  rating
0           2000                  Memento     8.4
1           2002      Catch Me If You Can     8.1
2           2005            Batman Begins     8.2
3           2006             The Departed     8.5
4           2006             The Prestige     8.5
5           2008          The Dark Knight     9.0
6           2010                Inception     8.8
7           2010           Shutter Island     8.2
8           2012         Django Unchained     8.4
9           2012    The Dark Knight Rises     8.4
10          2013  The Wolf of Wall Street     8.2
11          2014             Interstellar     8.6
```

In [6]:
list_movies_with_dicaprio_or_nolan =\
"""
-- SQL 查詢語法起點
SELECT movies.release_year,
       movies.title,
       movies.rating
  FROM actors
  JOIN casting
    ON actors.id = casting.actor_id
  JOIN top_rated_movies AS movies
    ON casting.movie_id = movies.id
 WHERE actors.name = 'Leonardo DiCaprio' OR
       movies.director = 'Christopher Nolan'
 GROUP BY movies.title
 ORDER BY movies.release_year;
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All.

In [7]:
class TestUnionJoin(unittest.TestCase):
    def test_01_summarize_party_votes_percentages(self):
        party_votes_percentages = pd.read_sql(summarize_party_votes_percentages, conn)
        self.assertEqual(party_votes_percentages.shape, (9, 3))
        np.testing.assert_equal(party_votes_percentages['election'].values,
                               np.array(['不分區立委', '不分區立委', '不分區立委', '區域立委', '區域立委', '區域立委', '總統副總統', '總統副總統', '總統副總統']))
        np.testing.assert_equal(party_votes_percentages['party'].values,
                               np.array(['中國國民黨', '民主進步黨', '親民黨', '中國國民黨', '民主進步黨', '親民黨', '中國國民黨', '民主進步黨', '親民黨']))
        np.testing.assert_equal(party_votes_percentages['votes_percentage'].values,
                               np.array([0.3336, 0.3398, 0.0366, 0.4071, 0.4511, 0.0043, 0.3861, 0.5713, 0.0426]))
    def test_02_list_lakers_players_ppg(self):
        lakers_players_ppg = pd.read_sql(list_lakers_players_ppg, conn)
        self.assertEqual(lakers_players_ppg.shape, (16, 4))
        np.testing.assert_equal(lakers_players_ppg['fullName'].unique(),
                               np.array(['Los Angeles Lakers']))
        np.testing.assert_equal(lakers_players_ppg['firstName'].values[:5],
                               np.array(['LeBron', 'Anthony', 'Kyle', 'Marc', 'Dennis']))
        np.testing.assert_equal(lakers_players_ppg['lastName'].values[:5],
                               np.array(['James', 'Davis', 'Kuzma', 'Gasol', 'Schroder']))
        np.testing.assert_equal(lakers_players_ppg['ppg'].values[:5],
                               np.array([27.0, 23.9, 15.4, 14.2, 14.1]))
    def test_03_list_ppg_leader_by_teams(self):
        ppg_leader_by_teams = pd.read_sql(list_ppg_leader_by_teams, conn)
        self.assertEqual(ppg_leader_by_teams.shape, (30, 3))
        np.testing.assert_equal(ppg_leader_by_teams['player'].values[:3],
                               np.array(['Trae Young', 'Kemba Walker', 'Kevin Durant']))
        np.testing.assert_equal(ppg_leader_by_teams['player'].values[-3:],
                               np.array(['Kyle Lowry', 'Donovan Mitchell', 'Russell Westbrook']))
        np.testing.assert_equal(ppg_leader_by_teams['ppg'].values[:3],
                               np.array([24.1, 19.8, 27.1]))
        np.testing.assert_equal(ppg_leader_by_teams['ppg'].values[-3:],
                               np.array([14.9, 22.9, 23.2]))
    def test_04_list_movies_in_which_tom_hanks_appeared(self):
        movies_in_which_tom_hanks_appeared = pd.read_sql(list_movies_in_which_tom_hanks_appeared, conn)
        self.assertEqual(movies_in_which_tom_hanks_appeared.shape, (6, 5))
        np.testing.assert_equal(movies_in_which_tom_hanks_appeared['release_year'].values,
                               np.array([1994, 1995, 1998, 1999, 2002, 2010]))
        np.testing.assert_equal(movies_in_which_tom_hanks_appeared['is_lead_actor'].values,
                               np.array([1, 1, 1, 1, 0, 1]))
        np.testing.assert_equal(movies_in_which_tom_hanks_appeared['title'].values,
                               np.array(['Forrest Gump', 'Toy Story', 'Saving Private Ryan', 'The Green Mile', 'Catch Me If You Can', 'Toy Story 3']))
    def test_05_list_movies_with_dicaprio_or_nolan(self):
        movies_with_dicaprio_or_nolan = pd.read_sql(list_movies_with_dicaprio_or_nolan, conn)
        self.assertEqual(movies_with_dicaprio_or_nolan.shape, (12, 3))
        self.assertTrue(set(['The Dark Knight', 'Inception', 'The Wolf of Wall Street', 'Interstellar']).issubset(movies_with_dicaprio_or_nolan['title'].values))

suite = unittest.TestLoader().loadTestsFromTestCase(TestUnionJoin)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)

test_01_summarize_party_votes_percentages (__main__.TestUnionJoin) ... ok
test_02_list_lakers_players_ppg (__main__.TestUnionJoin) ... ok
test_03_list_ppg_leader_by_teams (__main__.TestUnionJoin) ... ok
test_04_list_movies_in_which_tom_hanks_appeared (__main__.TestUnionJoin) ... ok
test_05_list_movies_with_dicaprio_or_nolan (__main__.TestUnionJoin) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.505s

OK


In [8]:
print("您在 {} 道 SQL 練習中答對了 {} 題。".format(number_of_test_runs, number_of_successes))

您在 5 道 SQL 練習中答對了 5 題。
